In [ ]:
# All imports are being called in this block
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
import matplotlib.pyplot as plt
import math

In [ ]:
# This function deals with loading the data and doing some minor operations
def load_and_preprocess_data(file_path, selected_columns):
    data = pd.read_csv(file_path)
    data = data[selected_columns]
    data = data.dropna()
    pressure_mean = data['Pressure'].mean() # Calculate the mean of the Pressure column
    print("Mean Pressure ", pressure_mean)
    print(data[data['Pressure']==0].count())
    data.loc[data['Pressure'] == 0, 'Pressure'] = pressure_mean # Replace all the rows with the mean value if the pressure is recorded as zero
    print(data[data['Pressure']==0].count())
    data = data.sort_values(by='Timestamp').reset_index(drop=True) # Sort the data as per the timestamp and drop the timestamp column
    data['Timestamp'] = range(1, len(data) + 1) # Assign Numeric Values to the timestamp column
    return data 

In [ ]:
# This function splits the data into feature set and target
def split_features_target(data, target_column):
    X = data.drop([target_column, 'Timestamp'], axis=1)
    y = data[target_column]
    timestamps = data['Timestamp']
    return X, y, timestamps

In [ ]:

def split_train_test(X, y, timestamps, test_size=0.2, random_state=420):
    X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(
        X, y, timestamps, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test, timestamps_train, timestamps_test

In [ ]:
# Function to scale the data
def scale_data(X_train, X_test, y_train, y_test):
    feature_scaler = StandardScaler()
    X_train_scaled = feature_scaler.fit_transform(X_train)
    X_test_scaled = feature_scaler.transform(X_test)

    target_scaler = MinMaxScaler()
    y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, feature_scaler, target_scaler

In [ ]:
# The model is built in this block of code
def build_model(input_dim, hidden_layer_sizes=(64, 128, 128, 64), learning_rate_init=0.004):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], input_dim=input_dim, activation='relu'))
    for layer_size in hidden_layer_sizes[1:]:
        model.add(Dense(layer_size, activation='relu'))
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_init)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

In [ ]:
#This class has a function on_epoch_end which stores the mse, mae, r2 and rmse on the end of each epoch. 

class MetricsHistory(Callback):
    def __init__(self, X_test, y_test, timestamps_test):
        super().__init__()
        self.X_test = X_test
        self.y_test = y_test
        self.timestamps_test = timestamps_test
        self.mse_history = []
        self.r2_history = []
        self.mae_history = []
        self.rmse_history = []
        self.y_pred = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_test).flatten()
        self.y_pred = y_pred
        mse = mean_squared_error(self.y_test, y_pred)
        mae = mean_absolute_error(self.y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(self.y_test, y_pred)
        self.mse_history.append(mse)
        self.mae_history.append(mae)
        self.rmse_history.append(rmse)
        self.r2_history.append(r2)

In [ ]:
def train_and_record_metrics(X_train_scaled, y_train, X_test_scaled, y_test, timestamps_test, input_dim, hidden_layer_sizes, learning_rate, batch_size):
    model = build_model(input_dim=input_dim, hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=learning_rate) # Initializimg the model with the given parameters
    metrics_history = MetricsHistory(X_test_scaled, y_test, timestamps_test)  # Making the object of the class MetricsHistory
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1) # It is used for adjusting the learningn rate with increasing epocs. It uses val_loss as its metrics by which it judges whether is it needed to change the learning rate or not
    
    print(f"Training with batch size: {batch_size}")
    model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
              epochs=300, batch_size=batch_size, callbacks=[metrics_history, lr_scheduler], verbose=0) # Training the model
    
    # Calculate the number of batches
    num_batches_train = math.ceil(len(X_train_scaled) / batch_size)
    num_batches_test = math.ceil(len(X_test_scaled) / batch_size)
    print(f"Number of batches (train): {num_batches_train}")
    print(f"Number of batches (test): {num_batches_test}")
    
    # Calculate final metrics
    y_pred = model.predict(X_test_scaled).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    return metrics_history, model, mse, mae, rmse, r2

In [ ]:
# This function is plotting the MSE, MAE, RMSE and R2 from the metrics_histories stored for each model trained with different type of input.
def plot_metrics(metrics_histories, labels, fontsize=14):
    plt.figure(figsize=(20, 12))
    
    plt.subplot(2, 2, 1)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.mse_history, label=f'{label}')
    plt.xlabel('Epoch', fontsize=fontsize)
    plt.ylabel('MSE', fontsize=fontsize)
    plt.title('MSE vs Epochs', fontsize=fontsize)
    plt.legend(fontsize=20)
    
    plt.subplot(2, 2, 2)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.mae_history, label=f'{label}')
    plt.xlabel('Epoch', fontsize=fontsize)
    plt.ylabel('MAE', fontsize=fontsize)
    plt.title('MAE vs Epochs', fontsize=fontsize)
    plt.legend(fontsize=20)
    
    plt.subplot(2, 2, 3)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.rmse_history, label=f'{label}')
    plt.xlabel('Epoch', fontsize=fontsize)
    plt.ylabel('RMSE', fontsize=fontsize)
    plt.title('RMSE vs Epochs', fontsize=fontsize)
    plt.legend(fontsize=20)
    
    plt.subplot(2, 2, 4)
    for label, history in zip(labels, metrics_histories):
        plt.plot(history.r2_history, label=f'{label}')
    plt.xlabel('Epoch', fontsize=fontsize)
    plt.ylabel('R² Score', fontsize=fontsize)
    plt.title('R² vs Epochs', fontsize=fontsize)
    plt.legend(fontsize=20)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# This function is just plotting the predicted river level with the actual river level at the selected timestamps
def plot_predictions(timestamps, actual, predicted, title, sample_size=500):
    sample_indices = np.random.choice(len(timestamps), sample_size, replace=False)
    sampled_timestamps = np.array(timestamps)[sample_indices]
    sampled_actual = np.array(actual)[sample_indices]
    sampled_predicted = np.array(predicted)[sample_indices]

    sorted_indices = np.argsort(sampled_timestamps)
    sorted_timestamps = sampled_timestamps[sorted_indices]
    sorted_actual = sampled_actual[sorted_indices]
    sorted_predicted = sampled_predicted[sorted_indices]
    
    plt.figure(figsize=(15, 8))
    plt.plot(sorted_timestamps, sorted_actual, label='Actual')
    plt.plot(sorted_timestamps, sorted_predicted, label='Predicted')
    plt.xlabel('Timestamp')
    plt.ylabel('Level')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
def ablation_feature_study(selected_columns, target_column='Level'):
    results_df = pd.DataFrame(columns=['Features', 'MSE', 'MAE', 'RMSE', 'R²'])

    merged_df = load_and_preprocess_data('./Datasets/2021-Kippure.csv', selected_columns) #Calling the function to do some preprocessing and loading the data

    original_X, original_y, timestamps = split_features_target(merged_df, target_column) # Splitting the data into features, target and timestamps. Timestamp has nothing to do with the logic. Actually, I wanted to see that for a particular timestamp, what is the river level and what is the predicted river level. So I have taken timestamps column into account this time.
    original_X_train, original_X_test, original_y_train, original_y_test, timestamps_train, timestamps_test = split_train_test(
        original_X, original_y, timestamps) # Calling the function to split the data into training and testing data. 
    original_X_train_scaled, original_X_test_scaled, original_y_train_scaled, original_y_test_scaled, feature_scaler, target_scaler = scale_data(
        original_X_train, original_X_test, original_y_train, original_y_test) # Calling the function to scale the data. We have used StandardScaler and MinmaxScaler to scale the data. 
#  Defined the model parameters below
    learning_rate = 0.001
    hidden_layer_sizes = (64, 128, 128, 64)
    batch_size = 256
    metrics_histories = []
    labels = []
# Calling the function to train the model using all the features of the feature set as input. 
    metrics_history, model, mse, mae, rmse, r2 = train_and_record_metrics(original_X_train_scaled, original_y_train_scaled, original_X_test_scaled,
                                                      original_y_test_scaled, timestamps_test, original_X.shape[1], hidden_layer_sizes, learning_rate, batch_size)
    metrics_histories.append(metrics_history)
    labels.append("All Features")

    results_df = pd.concat([results_df, pd.DataFrame({'Features': ['All Features'], 'MSE': [mse], 'MAE': [mae], 'RMSE': [rmse], 'R²': [r2]})]) # Adding the result to a dataframe.

    print(f"All Features - MSE: {mse}, MAE: {mae}, RMSE: {rmse}, R²: {r2}")
    plot_predictions(timestamps_test, original_y_test_scaled, metrics_history.y_pred, "Predicted vs Actual with All Features") # This function is called to plot the actual river level and the predicted river level for some instances
# Loop for the feature ablation
    for features in [['Windspeed'], ['Windspeed', 'Wind direction'], ['Windspeed', 'Wind direction', 'Rainfall'],['Windspeed', 'Humidity', 'Rainfall'],['Windspeed', 'Wind direction','Humidity', 'Rainfall']]:
        ablated_columns = [col for col in selected_columns if col not in features] # Removing the above mentioned columns from the selected columns. 
        merged_df = load_and_preprocess_data('./Datasets/2021-Kippure.csv', ablated_columns) #Now we will be considering ablated_columns for training the model

        X, y, timestamps = split_features_target(merged_df, target_column)
        X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = split_train_test(X, y, timestamps)
        X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, feature_scaler, target_scaler = scale_data(X_train, X_test, y_train, y_test)

        metrics_history, model, mse, mae, rmse, r2 = train_and_record_metrics(X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, timestamps_test,
                                                          X.shape[1], hidden_layer_sizes, learning_rate, batch_size)
        metrics_histories.append(metrics_history) # This will be having the history of each iteration
        labels.append(f"Without {', '.join(features)}")

        results_df = pd.concat([results_df, pd.DataFrame({'Features': [f"Without {', '.join(features)}"], 'MSE': [mse], 'MAE': [mae], 'RMSE': [rmse], 'R²': [r2]})])

        print(f"Without {', '.join(features)} - MSE: {mse}, MAE: {mae}, RMSE: {rmse}, R²: {r2}")
        plot_predictions(timestamps_test, y_test_scaled, metrics_history.y_pred, f"Predicted vs Actual without {', '.join(features)}") # Called this funtion to plot the actual river level and the predicted river level for some time stamps

    print(results_df)
    plot_metrics(metrics_histories, labels) # This function is called to plot the variation of mse, mae, r2 etc. with each epoch

In [ ]:
if __name__ == "__main__":
    selected_columns = ['Windspeed', 'Humidity', 'Temperature', 'Dewpoint', 'Pressure', 'Reading', 'Wind direction', 'Level', 'Timestamp']

    print("Performing Feature Ablation Study...")
    ablation_feature_study(selected_columns) # Called the feature ablation function with the selected columns